### Utilizar el archivo ratings_libros_usuario_tarea para agregar dichos ratings al archivo de ratings csv. 

In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np

In [2]:
nombre_zip = "goodbooks-20210115T022509Z-001.zip"

In [3]:
with ZipFile(nombre_zip, 'r') as zip: 
    zip.printdir() 

File Name                                             Modified             Size
goodbooks/                                     2021-01-14 21:59:16            0
goodbooks/books.csv                            2019-09-20 20:05:38      3296741
goodbooks/book_tags.csv                        2019-09-20 20:05:36     16665883
goodbooks/ratings.csv                          2019-09-20 20:05:38     12437766
goodbooks/ratings_libros_usuario_tarea.ods     2021-01-08 18:22:34        23407
goodbooks/ratings_libros_usuario_tarea.xlsx    2021-01-14 21:57:56        23165
goodbooks/sample_book.xml                      2019-09-20 20:05:40        28402
goodbooks/tags.csv                             2019-09-20 20:05:40       722480
goodbooks/to_read.csv                          2019-09-20 20:05:40      9407334


In [4]:
archivo_zip = ZipFile(nombre_zip, 'r')

In [5]:
ratings_zip = archivo_zip.open("goodbooks/ratings.csv")
ratings = pd.read_csv(ratings_zip)

In [6]:
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [7]:
ratings_usuario_zip = archivo_zip.open("goodbooks/ratings_libros_usuario_tarea.xlsx")
ratings_usuario = pd.read_excel(ratings_usuario_zip, sheet_name = "ratings", usecols = ["title", "rating"])
ratings_usuario.head()

,title,rating
0,El Avaro,it was amazing
1,Estudio en escarlata,it was amazing
2,El principito,it was amazing
3,Cuando Hitler robó el conejo rosa,it was amazing
4,The Metamorphosis,it was amazing


In [8]:
ratings_catalogo = pd.read_excel(ratings_usuario_zip, sheet_name = "catálogo")
ratings_catalogo = ratings_catalogo.set_index("rating")
ratings_catalogo

,estrellas
rating,
did not like it,1
it was ok,2
liked it,3
really liked it,4
it was amazing,5


In [9]:
#Mapeamos el rating de este diccionario
ratings_catalogo = ratings_catalogo.to_dict()["estrellas"]
ratings_usuario["rating"] = ratings_usuario["rating"].map(ratings_catalogo)

In [10]:
ratings

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


In [11]:
#Cargamos el archivo que contiene el id y nombre de los libros 
libros_zip = archivo_zip.open("goodbooks/books.csv")
libros = pd.read_csv(libros_zip, usecols = ["book_id", "title"], encoding = "utf-8")

In [12]:
#Quitamos duplicados y creamos un diccionario para mapear sobre la columna
#de title de ratings_usuario y obtener el id de la película
libros = libros.drop_duplicates()
libros = libros.set_index("title")
libros_dict = libros.to_dict()["book_id"]

In [13]:
ratings_usuario["title_id"] = ratings_usuario["title"].map(libros_dict)

In [14]:
#Hay películas que el diccionario no mapeo, por lo tanto son NaN
ratings_usuario["title_id"].isna().sum()

78

In [15]:
con_nan = ratings_usuario[ratings_usuario.title_id.isna()]

In [16]:
libros_con_nan = set(con_nan["title"])

In [17]:
print(libros_con_nan)

{'A la sombra del ángel', 'La historiadora', 'Harry Potter y el legado maldito. Partes uno y dos', 'Fuego y sangre (La historia de los Targaryen, #1)', 'Estudio en escarlata', 'Un mundo sin fin', 'Donde habitan los ángeles', 'El resplandor', 'Nada se opone a la noche', 'Un mal nombre (La amiga estupenda, #2)', "La amiga estupenda (L'amica geniale #1)", 'Cometas en el cielo', 'Las Mil y Una Noches', 'La verdad sobre el caso Harry Quebert', 'Usted puede sanar su vida', 'Muchas vidas, muchos maestros', 'Huesos de lagartija', 'El nombre de la rosa', 'El llano en llamas', 'Fantastic Beasts and Where to Find Them: The Original Screenplay (Fantastic Beasts: The Original Screenplay, #1)', 'Basada en hechos reales', 'Mi negro pasado (Como agua para chocolate, #3)', 'El Avaro', 'The Shadow of the Wind (The Cemetery of Forgotten Books, #1)', 'Viaje Al Centro de la Tierra', 'La ridícula idea de no volver a verte', 'Juan Salvador Gaviota', 'El diablo de los números', 'Jaque al Psicoanalista (El psi

In [18]:
def DistJaccard(str1, str2):
    str1 = set(str1.split())
    str2 = set(str2.split())
    return float(len(str1 & str2)) / len(str1 | str2)

In [19]:
arr = []
for titulo in list(set(libros.index)):
    arr.append(DistJaccard(titulo,"Fantastic Beasts - The Crimes of Grindelwald: The Original Screenplay"))

In [20]:
arr = np.array(arr)

In [21]:
arr

array([0.        , 0.07142857, 0.        , ..., 0.        , 0.        ,
       0.08333333])

In [22]:
indice = np.where(arr == max(arr))[0][0]

In [23]:
lista_libros = list(set(libros.index))

In [24]:
lista_libros[indice]

'Fantastic Beasts and Where to Find Them: The Original Screenplay'

In [25]:
def buscar(libro_a_buscar):
    arr = []
    for titulo in list(set(libros.index)):
        arr.append(DistJaccard(titulo,libro_a_buscar))
    arr = np.array(arr)
    indice = np.where(arr == max(arr))[0][0]
    return [libro_a_buscar, lista_libros[indice], float(max(arr))]

In [26]:
resultados = []
for libro in libros_con_nan:
    resultados.append(buscar(libro_a_buscar = libro))

In [27]:
resultado_mayor_umbral = []
for i in range(len(resultados)):
    resultado_mayor_umbral.append(resultados[i][2] > 0.3)

In [28]:
np.array(resultados)[resultado_mayor_umbral]

array([['La historiadora', 'La tregua', '0.3333333333333333'],
       ['El resplandor', 'El Deafo', '0.3333333333333333'],
       ['Fantastic Beasts and Where to Find Them: The Original Screenplay (Fantastic Beasts: The Original Screenplay, #1)',
        'Fantastic Beasts and Where to Find Them: The Original Screenplay',
        '0.7142857142857143'],
       ['El Avaro', 'El Deafo', '0.3333333333333333'],
       ['The Shadow of the Wind (The Cemetery of Forgotten Books, #1)',
        'The Shadow of the Wind (The Cemetery of Forgotten Books,  #1)',
        '1.0'],
       ['El principito', 'El Deafo', '0.3333333333333333'],
       ['La casa de los espíritus', 'La casa de Bernarda Alba',
        '0.42857142857142855'],
       ['Voices from Chernobyl: The Oral History of a Nuclear Disaster',
        'The History of Tom Jones, a Foundling', '0.3076923076923077'],
       ['Ready Player One (Ready Player One, #1)', 'Ready Player One',
        '0.5'],
       ['The Pillars of the Earth (Kingsbr

Podemos observar que con la distancia de Jaccard con un umbral de 0.3 sólo algunos títulos coinciden con los que verdaderamente son y corresponden, sin embargo hay algunos títulos de libros que no corresponden al que está en catálogo, ejemplo: La Bbúsqueda y La Tregua (tienen un indice alto de Jaccard por que los titulso se componen de dos palabras). Por lo tanto, creamos un nuevo ID para estas películas.

### Asignar un nuevo id de usuario a los ratings agregados en el paso anterior.

In [29]:
libros_zip = archivo_zip.open("goodbooks/books.csv")
libros = pd.read_csv(libros_zip, usecols = ["book_id", "title"])
libros

,book_id,title
0,2767052,"The Hunger Games (The Hunger Games, #1)"
1,3,Harry Potter and the Sorcerer's Stone (Harry P...
2,41865,"Twilight (Twilight, #1)"
3,2657,To Kill a Mockingbird
4,4671,The Great Gatsby
...,...,...
9995,7130616,"Bayou Moon (The Edge, #2)"
9996,208324,"Means of Ascent (The Years of Lyndon Johnson, #2)"
9997,77431,The Mauritius Command
9998,8565083,Cinderella Ate My Daughter: Dispatches from th...


In [30]:
ratings

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


In [31]:
ratings["user_id"].value_counts().sort_index()

1         3
2         3
3         2
4         3
5         5
         ..
53420     6
53421     8
53422    18
53423     2
53424    16
Name: user_id, Length: 53424, dtype: int64

In [32]:
ratings_usuario["user_id"] = 53425

In [33]:
ratings["book_id"].value_counts().sort_index()

1        100
2        100
3        100
4        100
5        100
        ... 
9996      98
9997      89
9998      95
9999      99
10000     96
Name: book_id, Length: 10000, dtype: int64

In [34]:
ratings_usuario["title_id"].isna().sum()

78

In [35]:
ratings_usuario.loc[ratings_usuario["title_id"].isna(), "title_id"] = [10000+x for x in range(ratings_usuario["title_id"].isna().sum())]

In [36]:
ratings_usuario_filt = ratings_usuario[["title_id", "user_id", "rating"]].rename(columns = {"title_id": "book_id"})

In [37]:
ratings

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


In [38]:
ratings_usuario_filt

,book_id,user_id,rating
0,10000.0,53425,5
1,10001.0,53425,5
2,10002.0,53425,5
3,10003.0,53425,5
4,485894.0,53425,5
...,...,...,...
101,10073.0,53425,2
102,10074.0,53425,2
103,10075.0,53425,2
104,10076.0,53425,2


In [39]:
ratings_actualizada = pd.concat([ratings, ratings_usuario_filt]).reset_index(drop=True)

In [40]:
ratings_actualizada

,book_id,user_id,rating
0,1.0,314,5
1,1.0,439,3
2,1.0,588,5
3,1.0,1169,4
4,1.0,1185,4
...,...,...,...
981857,10073.0,53425,2
981858,10074.0,53425,2
981859,10075.0,53425,2
981860,10076.0,53425,2


In [41]:
import pyspark

In [42]:
sc = pyspark.SparkContext(appName ="clase_datos_masivos")

In [43]:
from pyspark.sql import SparkSession

In [44]:
# local[*] usará todos los núcleos disponibles 
spark = SparkSession.builder\
                    .master('local[*]') \
                    .appName('first_spark') \
                    .getOrCreate()

In [45]:
print(spark)

In [46]:
# Importar funciones
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [47]:
ratings = spark.createDataFrame(ratings_actualizada)

In [48]:
ratings.show()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|    1.0|    314|     5|
|    1.0|    439|     3|
|    1.0|    588|     5|
|    1.0|   1169|     4|
|    1.0|   1185|     4|
|    1.0|   2077|     4|
|    1.0|   2487|     4|
|    1.0|   2900|     5|
|    1.0|   3662|     4|
|    1.0|   3922|     5|
|    1.0|   5379|     5|
|    1.0|   5461|     3|
|    1.0|   5885|     5|
|    1.0|   6630|     5|
|    1.0|   7563|     3|
|    1.0|   9246|     1|
|    1.0|  10140|     4|
|    1.0|  10146|     5|
|    1.0|  10246|     4|
|    1.0|  10335|     4|
+-------+-------+------+
only showing top 20 rows



In [49]:
# Convertir al tipo apropiado de datos
ratings = ratings.select(ratings.user_id.cast("integer"), ratings.book_id.cast("integer"), ratings.rating.cast("double"))

# Revisar que se haya aplicado
ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- rating: double (nullable = true)



In [50]:
ratings.createOrReplaceTempView("ratings")

In [51]:
spark.catalog.listTables()

[Table(name='ratings', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [52]:
ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- rating: double (nullable = true)



In [53]:
# Crean tabla de entrenamiento y prueba
(train, test) = ratings.randomSplit([0.75, 0.25], seed = 202012)

In [54]:
als_1 = ALS(userCol="user_id", itemCol="book_id", ratingCol="rating")

In [55]:
modelo_1 = als_1.fit(train)

In [56]:
predicciones_1 = modelo_1.transform(test)

In [57]:
predicciones_1.show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  10727|    148|   4.0|  3.717816|
|  26629|    148|   4.0|   3.74888|
|  28767|    148|   3.0|  3.025691|
|   9165|    148|   3.0| 3.7544518|
|  38475|    148|   4.0| 3.6370912|
|   7001|    148|   4.0| 3.7668166|
|  20967|    148|   3.0| 4.1432114|
|  14603|    148|   4.0| 3.5522707|
|  13407|    148|   4.0| 3.9852662|
|  42404|    148|   5.0| 3.8055043|
|   8440|    148|   3.0| 3.1180573|
|   9125|    148|   1.0| 2.1694806|
|   2171|    148|   4.0|  3.679798|
|  38798|    148|   3.0| 3.3506374|
|  24499|    148|   4.0|  2.851161|
|  51838|    148|   4.0| 3.7141058|
|  22602|    148|   3.0| 3.2579412|
|  44397|    148|   5.0|  4.167453|
|  47730|    148|   2.0|  4.044771|
|  18031|    148|   5.0|  4.088013|
+-------+-------+------+----------+
only showing top 20 rows



In [58]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [59]:
predicciones_1.select(avg("rating")).show()

+------------------+
|       avg(rating)|
+------------------+
|3.8535770792894777|
+------------------+



In [60]:
predicciones_1.select(avg("prediction")).show()

+---------------+
|avg(prediction)|
+---------------+
|            NaN|
+---------------+



In [61]:
predicciones_1.select("prediction").count()

245228

In [62]:
predicciones_1.createOrReplaceTempView("predicciones_1")

In [63]:
spark.catalog.listTables()

[Table(name='predicciones_1', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='ratings', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [64]:
query = " select * from predicciones_1 where prediction = 'NaN' "
spark.sql(query).show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|   5498|   9465|   5.0|       NaN|
|  40724|   9852|   5.0|       NaN|
|  35503|   3226|   3.0|       NaN|
|  22379|   6393|   4.0|       NaN|
|  10164|   6393|   4.0|       NaN|
|  28450|   9182|   5.0|       NaN|
|  53001|   9182|   5.0|       NaN|
|  21522|   6559|   5.0|       NaN|
|  51851|   9517|   5.0|       NaN|
|  45929|   6598|   3.0|       NaN|
|  42863|   7530|   4.0|       NaN|
|  16033|   7910|   5.0|       NaN|
|   8834|   8650|   4.0|       NaN|
|  47271|   8650|   4.0|       NaN|
|  10715|   8743|   4.0|       NaN|
|  24443|   8743|   4.0|       NaN|
|  13537|   8743|   3.0|       NaN|
|  46438|   8779|   3.0|       NaN|
|  27778|   9169|   2.0|       NaN|
|  44276|   3377|   3.0|       NaN|
+-------+-------+------+----------+
only showing top 20 rows



In [65]:
query = " select * from predicciones_1 order by prediction "
spark.sql(query).show()

+-------+-------+------+-------------+
|user_id|book_id|rating|   prediction|
+-------+-------+------+-------------+
|  11759|   6089|   5.0|   -0.5707712|
|  47325|   7636|   2.0|  -0.46481428|
|  44657|   6940|   5.0|  -0.25109816|
|  20460|   3075|   1.0|  -0.22632657|
|  40603|   8956|   3.0|  -0.12147481|
|  36949|   8748|   4.0|-0.0040562544|
|  34475|   7063|   4.0| -0.003401041|
|  51217|   9445|   5.0|  0.080078304|
|   5145|   9086|   3.0|   0.08718853|
|  50817|   9218|   1.0|   0.09673029|
|  14509|   8336|   4.0|   0.19717786|
|  12824|   7580|   5.0|   0.21134312|
|  34036|   5373|   2.0|    0.2575391|
|  17226|   8270|   5.0|    0.2629158|
|  11759|   3224|   4.0|    0.2672944|
|    448|   7735|   4.0|   0.27702954|
|  20014|   6481|   2.0|   0.30908883|
|  36165|   8617|   1.0|    0.3161653|
|  29877|   6034|   5.0|   0.31727648|
|   1582|   9967|   4.0|   0.33400908|
+-------+-------+------+-------------+
only showing top 20 rows



In [66]:
query = " select * from predicciones_1  where prediction <> 'NaN' order by prediction desc "
spark.sql(query).show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  48129|   5919|   5.0|  5.861641|
|  42047|   5919|   5.0| 5.8265243|
|   8911|   6089|   5.0| 5.8110967|
|  52469|    950|   5.0| 5.7814174|
|  21058|   7945|   5.0|  5.771408|
|  19171|    422|   5.0| 5.7588367|
|  11413|   4649|   5.0| 5.7397003|
|  37939|   1110|   5.0|  5.714551|
|  26547|   7818|   5.0|  5.711295|
|  51548|   5701|   5.0| 5.7024736|
|  50138|   1684|   5.0| 5.6939664|
|  18171|   9076|   5.0|  5.684693|
|  40566|   1788|   5.0| 5.6772146|
|  24744|   4911|   5.0| 5.6745973|
|  38654|   2507|   5.0| 5.6739907|
|  32852|   7834|   5.0|  5.672102|
|  24048|   8109|   5.0| 5.6553354|
|  34272|   6052|   5.0| 5.6474657|
|  16384|   2729|   5.0| 5.6474204|
|  35949|   3753|   5.0|   5.64336|
+-------+-------+------+----------+
only showing top 20 rows



In [67]:
train.createOrReplaceTempView("train")

In [68]:
spark.catalog.listTables()

[Table(name='predicciones_1', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='ratings', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='train', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [69]:
query = " select * from train where user_id = 53425"
spark.sql(query).show()

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|  53425|      2|   5.0|
|  53425|      5|   5.0|
|  53425|    343|   5.0|
|  53425|   4381|   5.0|
|  53425|   9712|   5.0|
|  53425|  10001|   5.0|
|  53425|  10002|   5.0|
|  53425|  10003|   5.0|
|  53425|  10004|   5.0|
|  53425|  10005|   5.0|
|  53425|  10008|   5.0|
|  53425|  10009|   5.0|
|  53425|  10011|   5.0|
|  53425|  10012|   5.0|
|  53425|  10013|   5.0|
|  53425|  10014|   5.0|
|  53425|  10015|   5.0|
|  53425|  10018|   5.0|
|  53425|  10021|   5.0|
|  53425|  10023|   5.0|
+-------+-------+------+
only showing top 20 rows



In [70]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [71]:
RMSE_1 = evaluator.evaluate(predicciones_1)
print(RMSE_1)

nan


In [72]:
predicciones_pandas = predicciones_1.toPandas()

In [73]:
predicciones_pandas.head()

,user_id,book_id,rating,prediction
0,10727,148,4.0,3.717816
1,26629,148,4.0,3.748880
2,28767,148,3.0,3.025691
3,9165,148,3.0,3.754452
4,38475,148,4.0,3.637091


In [74]:
predicciones_pandas[(predicciones_pandas["user_id"] == 53425) & (predicciones_pandas["prediction"].notnull())]

,user_id,book_id,rating,prediction
63337,53425,3,5.0,3.412700
71711,53425,10000,5.0,4.207197


In [77]:
als_2 = ALS(userCol="user_id", itemCol="book_id", ratingCol="rating", rank =15, maxIter =20, regParam =0.1,
          coldStartStrategy="drop", nonnegative =True, implicitPrefs = False)

In [78]:
# Generar modelo
modelo_2 = als_2.fit(train)

In [79]:
# Generar predicciones
predicciones_2 = modelo_2.transform(test)

In [80]:
predicciones_2.show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  10727|    148|   4.0| 3.3814664|
|  26629|    148|   4.0| 3.9016194|
|  28767|    148|   3.0| 2.9804752|
|   9165|    148|   3.0| 3.6830242|
|  38475|    148|   4.0| 3.4808042|
|   7001|    148|   4.0| 3.8477457|
|  20967|    148|   3.0| 3.8371656|
|  14603|    148|   4.0| 3.7016072|
|  13407|    148|   4.0|  3.862438|
|  42404|    148|   5.0| 3.8889074|
|   8440|    148|   3.0|   2.97492|
|   9125|    148|   1.0|  2.463397|
|   2171|    148|   4.0|  3.555404|
|  38798|    148|   3.0| 3.3011656|
|  24499|    148|   4.0| 2.8905485|
|  51838|    148|   4.0| 3.7496614|
|  22602|    148|   3.0| 3.3443098|
|  44397|    148|   5.0| 4.0911465|
|  47730|    148|   2.0|  3.770562|
|  18031|    148|   5.0|  4.024829|
+-------+-------+------+----------+
only showing top 20 rows



In [81]:
predicciones_pandas2 = predicciones_2.toPandas()

In [82]:
predicciones_pandas2[(predicciones_pandas2["user_id"] == 53425) & (predicciones_pandas2["prediction"].notnull())]

,user_id,book_id,rating,prediction
62978,53425,3,5.0,3.398588
71312,53425,10000,5.0,3.975679
